In [2]:
from __future__ import print_function
import pandas as pd
import numpy as np
import datetime as dt
#import quandl

# Data - This part doesn't work any more because quandel is defunct

In [4]:
datadir='../data/'
# assets=pd.read_csv(datadir+'SP500.txt', comment='#').set_index('Symbol')

# QUANDL={
#     ## Get a key (free) from quandl.com and copy it here
#     'authtoken':"skyJ_Rgz8mgzTuZJ7Biy",
#     'start_date':dt.date(2007, 1, 1),
#     'end_date':dt.date(2016, 12, 31)
# }
# RISK_FREE_SYMBOL = "USDOLLAR"
# data={}

#### Download loop

If it stops because of Quandl error codes 503 or 504, try re-running it (it won't download data already downloaded). If Quandl complains about the speed of requests, try adding sleep time.

In [4]:
# # download assets' data
# from time import sleep

# for ticker in assets.index:
#     if ticker in data:
#         continue
#     print('downloading %s from %s to %s' %(ticker, QUANDL['start_date'], QUANDL['end_date']))
#     try:
#         data[ticker] = quandl.get(assets.Quandlcode[ticker], **QUANDL)
#         #sleep(0.1)
#     except quandl.NotFoundError:
#         print('\tInvalid asset code')

downloading MMM from 2007-01-01 to 2016-12-31
downloading ABT from 2007-01-01 to 2016-12-31
downloading ABBV from 2007-01-01 to 2016-12-31
downloading ACN from 2007-01-01 to 2016-12-31
downloading ACE from 2007-01-01 to 2016-12-31
downloading ATVI from 2007-01-01 to 2016-12-31
downloading ADBE from 2007-01-01 to 2016-12-31
downloading ADT from 2007-01-01 to 2016-12-31
downloading AAP from 2007-01-01 to 2016-12-31
downloading AES from 2007-01-01 to 2016-12-31
downloading AET from 2007-01-01 to 2016-12-31
downloading AFL from 2007-01-01 to 2016-12-31
downloading AMG from 2007-01-01 to 2016-12-31
downloading A from 2007-01-01 to 2016-12-31
downloading GAS from 2007-01-01 to 2016-12-31
downloading APD from 2007-01-01 to 2016-12-31
downloading ARG from 2007-01-01 to 2016-12-31
downloading AKAM from 2007-01-01 to 2016-12-31
downloading AA from 2007-01-01 to 2016-12-31
downloading AGN from 2007-01-01 to 2016-12-31
downloading ALXN from 2007-01-01 to 2016-12-31
downloading ALLE from 2007-01-01

#### Computation 

In [23]:
# keys=[el for el in assets.index if not el in (set(assets.index)-set(data.keys()))]

# def select_first_valid_column(df, columns):
#     for column in columns:
#         if column in df.columns:
#             return df[column]
        
# # extract prices
# prices=pd.DataFrame.from_items([(k,select_first_valid_column(data[k], ["Adj. Close", "Close", "Value"])) 
#                                 for k in keys])

# #compute sigmas
# open_price=pd.DataFrame.from_items([(k,select_first_valid_column(data[k], ["Open"])) for k in keys])
# close_price=pd.DataFrame.from_items([(k,select_first_valid_column(data[k], ["Close"])) for k in keys])
# sigmas = np.abs(np.log(open_price.astype(float))-np.log(close_price.astype(float)))

# # extract volumes
# volumes=pd.DataFrame.from_items([(k,select_first_valid_column(data[k], ["Adj. Volume", "Volume"])) for k in keys])

# # fix risk free
# prices[RISK_FREE_SYMBOL]=10000*(1 + prices[RISK_FREE_SYMBOL]/(100*250)).cumprod()

#### Filtering 

In [24]:
# # filter NaNs - threshold at 2% missing values
# bad_assets = prices.columns[prices.isnull().sum()>len(prices)*0.02]
# if len(bad_assets):
#     print('Assets %s have too many NaNs, removing them' % bad_assets)

# prices = prices.loc[:,~prices.columns.isin(bad_assets)]
# sigmas = sigmas.loc[:,~sigmas.columns.isin(bad_assets)]
# volumes = volumes.loc[:,~volumes.columns.isin(bad_assets)]

# nassets=prices.shape[1]

# # days on which many assets have missing values
# bad_days1=sigmas.index[sigmas.isnull().sum(1) > nassets*.9]
# bad_days2=prices.index[prices.isnull().sum(1) > nassets*.9]
# bad_days3=volumes.index[volumes.isnull().sum(1) > nassets*.9]
# bad_days=pd.Index(set(bad_days1).union(set(bad_days2)).union(set(bad_days3))).sort_values()
# print ("Removing these days from dataset:")
# print(pd.DataFrame({'nan price':prices.isnull().sum(1)[bad_days],
#                     'nan volumes':volumes.isnull().sum(1)[bad_days],
#                     'nan sigmas':sigmas.isnull().sum(1)[bad_days]}))

# prices=prices.loc[~prices.index.isin(bad_days)]
# sigmas=sigmas.loc[~sigmas.index.isin(bad_days)]
# volumes=volumes.loc[~volumes.index.isin(bad_days)]

# # extra filtering
# print(pd.DataFrame({'remaining nan price':prices.isnull().sum(),
#                     'remaining nan volumes':volumes.isnull().sum(),
#                     'remaining nan sigmas':sigmas.isnull().sum()}))
# prices=prices.fillna(method='ffill')
# sigmas=sigmas.fillna(method='ffill')
# volumes=volumes.fillna(method='ffill')
# print(pd.DataFrame({'remaining nan price':prices.isnull().sum(),
#                     'remaining nan volumes':volumes.isnull().sum(),
#                     'remaining nan sigmas':sigmas.isnull().sum()}))

Assets Index(['ABBV', 'ACE', 'ADT', 'GAS', 'ARG', 'AA', 'ALLE', 'GOOG', 'ALTR',
       'AVGO', 'BXLT', 'BRCM', 'CVC', 'CAM', 'CPGX', 'CMCSK', 'DLPH', 'DAL',
       'DFS', 'DISCK', 'DG', 'DPS', 'EMC', 'FB', 'GM', 'HCA', 'HCBK', 'GMCR',
       'KMI', 'KHC', 'LYB', 'MNK', 'MPC', 'MHFI', 'MJN', 'WRK', 'KORS', 'NAVI',
       'NWSA', 'NWS', 'NLSN', 'PYPL', 'POM', 'PM', 'PSX', 'PCL', 'PPG', 'PCP',
       'QRVO', 'SNDK', 'SNI', 'SIAL', 'HOT', 'TEL', 'TE', 'TDC', 'TRV', 'TWC',
       'TRIP', 'TYC', 'UA', 'VRSK', 'V', 'XYL', 'ZTS'],
      dtype='object') have too many NaNs, removing them
Removing these days from dataset:
            nan price  nan sigmas  nan volumes
2007-01-02        438         NaN          NaN
2007-04-06        438         NaN          NaN
2010-04-02        438         NaN          NaN
2012-04-06        438         NaN          NaN
2012-10-29        438         NaN          NaN
2015-04-03        438         NaN          NaN
          remaining nan price  remaining nan sigmas 

#### Save 

In [ ]:
# # make volumes in dollars
# volumes = volumes*prices

# # compute returns
# returns = (prices.diff()/prices.shift(1)).fillna(method='ffill').ix[1:]

# bad_assets = returns.columns[((-.5>returns).sum()>0)|((returns > 2.).sum()>0)]
# if len(bad_assets):
#     print('Assets %s have dubious returns, removed' % bad_assets)
    
# prices = prices.loc[:,~prices.columns.isin(bad_assets)]
# sigmas = sigmas.loc[:,~sigmas.columns.isin(bad_assets)]
# volumes = volumes.loc[:,~volumes.columns.isin(bad_assets)]
# returns = returns.loc[:,~returns.columns.isin(bad_assets)]

# # remove USDOLLAR
# prices = prices.iloc[:,:-1]
# sigmas = sigmas.iloc[:,:-1]
# volumes = volumes.iloc[:,:-1]


# # save data
# prices.to_csv(datadir+'prices.csv.gz', compression='gzip', float_format='%.3f')
# volumes.to_csv(datadir+'volumes.csv.gz', compression='gzip', float_format='%d')
# returns.to_csv(datadir+'returns.csv.gz', compression='gzip', float_format='%.3e')
# sigmas.to_csv(datadir+'sigmas.csv.gz', compression='gzip', float_format='%.3e')

Assets Index(['AIG', 'ETFC', 'GGP', 'GNW', 'HIG', 'STT', 'XL'], dtype='object') have dubious returns, removed


# Instead we load data that was saved in 2016 

# Estimates 

In [13]:
prices = pd.read_csv(datadir+'prices.csv.gz', index_col=0, parse_dates=[0])
volumes = pd.read_csv(datadir+'volumes.csv.gz', index_col=0, parse_dates=[0])
returns = pd.read_csv(datadir+'returns.csv.gz', index_col=0, parse_dates=[0])
sigmas = pd.read_csv(datadir+'sigmas.csv.gz', index_col=0, parse_dates=[0])

In [14]:
# print("typical variance of returns: %g"%returns.var().mean())

In [15]:
sigma2_n=0.02
sigma2_r=0.0005

np.random.seed(0)
noise=pd.DataFrame(index=returns.index, columns=returns.columns, 
                   data=np.sqrt(sigma2_n)*np.random.randn(*returns.values.shape))
return_estimate= (returns + noise)*sigma2_r/(sigma2_r+sigma2_n)
return_estimate.USDOLLAR = returns.USDOLLAR

return_estimate.to_csv(datadir+'return_estimate.csv.gz', compression='gzip', float_format='%.3e')

In [18]:
agree_on_sign=np.sign(returns.iloc[:,:-1]) == np.sign(return_estimate.iloc[:,:-1])
print("Return predictions have the right sign %.1f%% of the times"%
      (100*agree_on_sign.sum().sum()/(agree_on_sign.shape[0]*(agree_on_sign.shape[1]-1))))

Return predictions have the right sign 53.8% of the times


In [19]:
volume_estimate=volumes.rolling(window=10, center=False).mean().dropna()
volume_estimate.to_csv(datadir+'volume_estimate.csv.gz', compression='gzip', float_format='%d')
sigmas_estimate = sigmas.rolling(window=10, center=False).mean().dropna()
sigmas_estimate.to_csv(datadir+'sigma_estimate.csv.gz', compression='gzip', float_format='%.3e')

# Risk model 

In [22]:
pd.read_csv(datadir + 'exposures.csv.gz', index_col=[0,1])

MMM       ABT       ACN      ATVI      ADBE       AAP   
2012-01-01 0  -0.015700  0.018570 -0.001942 -0.004899  0.029570  0.000272  \
           1   0.014410  0.001936 -0.011860  0.011070 -0.003846  0.037940   
           2  -0.011940 -0.014470 -0.016300 -0.029890 -0.018350  0.035050   
           3   0.033050  0.016800 -0.007668 -0.001364  0.023300 -0.030100   
           4  -0.006390  0.011110  0.018100 -0.032270  0.036930 -0.016720   
...                 ...       ...       ...       ...       ...       ...   
2016-12-01 10 -0.007958 -0.023560 -0.027300 -0.076180 -0.034560  0.022430   
           11  0.035700 -0.037500  0.032490  0.020280 -0.004009 -0.000561   
           12 -0.018130  0.005225 -0.010570 -0.031710 -0.015530 -0.008148   
           13  0.022590  0.044650  0.037900  0.052610  0.046280  0.030380   
           14  0.032830  0.043480  0.040180  0.037410  0.040650  0.034930   

                    AES       AET       AFL       AMG  ...       WYN   
2012-01-01 0   0.041490  0.070210  0.009169  0.028040  ...  0.042740  \
           1  -0.005988  0.019780  0.019210  0.011190  ... -0.005886   
           2  -0.032980  0.018810 -0.029360  0.006567  ...  0.010740   
           3   0.030640 -0.048440  0.000062 -0.068460  ... -0.006486   
           4   0.000108  0.009037  0.001585  0.000535  ... -0.008355   
...                 ...       ...       ...       ...  ...       ...   
2016-12-01 10 -0.002081 -0.020930 -0.023810 -0.002707  ...  0.037180   
           11 -0.030100 -0.095600  0.034340  0.028340  ... -0.013490   
           12 -0.094910  0.006471  0.006904  0.063390  ...  0.005232   
           13 -0.007570  0.044100  0.021410  0.012920  ...  0.030160   
           14  0.048940  0.041840  0.037380  0.074150  ...  0.045710   

                   WYNN       XEL       XRX      XLNX      YHOO       YUM   
2012-01-01 0   0.009461  0.014890 -0.006300 -0.006642 -0.049260  0.002671  \
           1   0.028280 -0.017790 -0.022270  0.003577  0.048470  0.007578   
           2   0.004300 -0.015040 -0.005466 -0.002203  0.054090  0.024970   
           3  -0.076830  0.042480  0.036080 -0.031170 -0.032450  0.008340   
           4  -0.030880  0.008445 -0.002662 -0.032790  0.020650  0.034310   
...                 ...       ...       ...       ...       ...       ...   
2016-12-01 10  0.069380  0.003751  0.017780 -0.021560 -0.048260  0.001527   
           11 -0.032330  0.006161  0.011490  0.000693 -0.030320 -0.003100   
           12 -0.002791 -0.123100 -0.000893 -0.009942  0.000517 -0.014070   
           13 -0.008824  0.032210  0.010530  0.025900  0.030170  0.040590   
           14  0.070450  0.013340  0.056280  0.042460  0.044910  0.040910   

                    ZBH      ZION  USDOLLAR  
2012-01-01 0   0.027390  0.091610  0.000007  
           1  -0.006017 -0.028540 -0.000005  
           2  -0.004025  0.006510  0.000001  
           3   0.010440  0.037960 -0.000002  
           4   0.011210  0.011810 -0.000001  
...                 ...       ...       ...  
2016-12-01 10 -0.034240 -0.015360  0.000003  
           11 -0.044070  0.046700  0.000010  
           12 -0.011480  0.105300 -0.000001  
           13  0.025040 -0.000699 -0.000002  
           14  0.036050  0.066320  0.000002  

[900 rows x 432 columns]

# The below computes risk models and broke because pandas dropped functionalities. I've re-run it to save multiindexes dataframes, but it's better to keep dictionaries. I'm not changing it any more because I don't want to pollute the repo with more data files. 

In [23]:
start_t="2012-01-01"
end_t="2016-12-31"

first_days_month=\
    pd.date_range(start=returns.index[next(i for (i,el) in enumerate(returns.index >= start_t) if el)-1],
                                 end=returns.index[-1], freq='MS')

k=15
exposures, factor_sigma, idyos = {}, {},{}
for day in first_days_month:
    used_returns = returns.loc[(returns.index < day)&
                        (returns.index >= day-pd.Timedelta("730 days"))]
    second_moment=used_returns.values.T@used_returns.values/used_returns.values.shape[0]
    eival,eivec=np.linalg.eigh(second_moment)
    factor_sigma[day]=np.diag(eival[-k:])
    exposures[day]=pd.DataFrame(data=eivec[:,-k:],index=returns.columns)
    idyos[day]=pd.Series(data=np.diag(eivec[:,:-k]@np.diag(eival[:-k])@eivec[:,:-k].T),index=returns.columns)
    
# pd.Panel(exposures).swapaxes(1,2).to_hdf(datadir+'risk_model.h5', 'exposures')
# pd.DataFrame(idyos).T.to_hdf(datadir+'risk_model.h5', 'idyos')
# pd.Panel(factor_sigma).to_hdf(datadir+'risk_model.h5', 'factor_sigma')

# load with
# store = pd.HDFStore('risk_model.h5')
# store.exposures, etc.

In [24]:
exposures

{Timestamp('2012-01-01 00:00:00'):                 0         1         2         3         4         5    
 MMM      -0.015695  0.014414 -0.011944  0.033046 -0.006390 -0.007830  \
 ABT       0.018575  0.001936 -0.014469  0.016797  0.011111 -0.022394   
 ACN      -0.001942 -0.011856 -0.016300 -0.007668  0.018103  0.036675   
 ATVI     -0.004899  0.011070 -0.029885 -0.001364 -0.032268 -0.012843   
 ADBE      0.029569 -0.003846 -0.018349  0.023303  0.036925  0.037608   
 ...            ...       ...       ...       ...       ...       ...   
 YHOO     -0.049259  0.048475  0.054087 -0.032455  0.020649 -0.025482   
 YUM       0.002671  0.007578  0.024972  0.008340  0.034309  0.006020   
 ZBH       0.027389 -0.006017 -0.004025  0.010440  0.011212 -0.035963   
 ZION      0.091610 -0.028535  0.006510  0.037963  0.011805  0.050844   
 USDOLLAR  0.000007 -0.000005  0.000001 -0.000002 -0.000001  0.000005   
 
                 6         7         8         9         10        11   
 MMM       0.04

# These are what are used elsewhere 

In [25]:
pd.read_csv(datadir + 'exposures.csv.gz', index_col=[0,1], parse_dates=[0])

MMM       ABT       ACN      ATVI      ADBE       AAP   
2012-01-01 0  -0.015700  0.018570 -0.001942 -0.004899  0.029570  0.000272  \
           1   0.014410  0.001936 -0.011860  0.011070 -0.003846  0.037940   
           2  -0.011940 -0.014470 -0.016300 -0.029890 -0.018350  0.035050   
           3   0.033050  0.016800 -0.007668 -0.001364  0.023300 -0.030100   
           4  -0.006390  0.011110  0.018100 -0.032270  0.036930 -0.016720   
...                 ...       ...       ...       ...       ...       ...   
2016-12-01 10 -0.007958 -0.023560 -0.027300 -0.076180 -0.034560  0.022430   
           11  0.035700 -0.037500  0.032490  0.020280 -0.004009 -0.000561   
           12 -0.018130  0.005225 -0.010570 -0.031710 -0.015530 -0.008148   
           13  0.022590  0.044650  0.037900  0.052610  0.046280  0.030380   
           14  0.032830  0.043480  0.040180  0.037410  0.040650  0.034930   

                    AES       AET       AFL       AMG  ...       WYN   
2012-01-01 0   0.041490  0.070210  0.009169  0.028040  ...  0.042740  \
           1  -0.005988  0.019780  0.019210  0.011190  ... -0.005886   
           2  -0.032980  0.018810 -0.029360  0.006567  ...  0.010740   
           3   0.030640 -0.048440  0.000062 -0.068460  ... -0.006486   
           4   0.000108  0.009037  0.001585  0.000535  ... -0.008355   
...                 ...       ...       ...       ...  ...       ...   
2016-12-01 10 -0.002081 -0.020930 -0.023810 -0.002707  ...  0.037180   
           11 -0.030100 -0.095600  0.034340  0.028340  ... -0.013490   
           12 -0.094910  0.006471  0.006904  0.063390  ...  0.005232   
           13 -0.007570  0.044100  0.021410  0.012920  ...  0.030160   
           14  0.048940  0.041840  0.037380  0.074150  ...  0.045710   

                   WYNN       XEL       XRX      XLNX      YHOO       YUM   
2012-01-01 0   0.009461  0.014890 -0.006300 -0.006642 -0.049260  0.002671  \
           1   0.028280 -0.017790 -0.022270  0.003577  0.048470  0.007578   
           2   0.004300 -0.015040 -0.005466 -0.002203  0.054090  0.024970   
           3  -0.076830  0.042480  0.036080 -0.031170 -0.032450  0.008340   
           4  -0.030880  0.008445 -0.002662 -0.032790  0.020650  0.034310   
...                 ...       ...       ...       ...       ...       ...   
2016-12-01 10  0.069380  0.003751  0.017780 -0.021560 -0.048260  0.001527   
           11 -0.032330  0.006161  0.011490  0.000693 -0.030320 -0.003100   
           12 -0.002791 -0.123100 -0.000893 -0.009942  0.000517 -0.014070   
           13 -0.008824  0.032210  0.010530  0.025900  0.030170  0.040590   
           14  0.070450  0.013340  0.056280  0.042460  0.044910  0.040910   

                    ZBH      ZION  USDOLLAR  
2012-01-01 0   0.027390  0.091610  0.000007  
           1  -0.006017 -0.028540 -0.000005  
           2  -0.004025  0.006510  0.000001  
           3   0.010440  0.037960 -0.000002  
           4   0.011210  0.011810 -0.000001  
...                 ...       ...       ...  
2016-12-01 10 -0.034240 -0.015360  0.000003  
           11 -0.044070  0.046700  0.000010  
           12 -0.011480  0.105300 -0.000001  
           13  0.025040 -0.000699 -0.000002  
           14  0.036050  0.066320  0.000002  

[900 rows x 432 columns]